# import

In [1]:
import os
import pickle
#pip install python-chess
import chess.pgn
from tqdm import tqdm
import pandas as pd
import psycopg2
from psycopg2 import sql


# Connect to DB

In [2]:
connection_params = {
    "dbname": "chessdb",
    "user": "postgres",    
    "password": "123456", # ALTER USER postgres PASSWORD '123456';
    "host": "localhost",
    "port": "5432"
}
conn = psycopg2.connect(**connection_params)
cur = conn.cursor()

# Set up table

In [3]:
# sudo -u postgres psql
# CREATE DATABASE chessdb;
table_name = "chess_positions"
create_table_query = f"""
DROP TABLE IF EXISTS {table_name};
CREATE TABLE IF NOT EXISTS {table_name} (
    id SERIAL PRIMARY KEY,
    position TEXT,
    result TEXT,
    white TEXT,
    black TEXT,
    next_move TEXT
);
"""
cur.execute(create_table_query)
conn.commit()
print(f"Table '{table_name}' is ready.")

Table 'chess_positions' is ready.


# save data to DB

In [4]:
def save_data_to_db(data):
    for row in data:
        insert_query = f"""
        INSERT INTO {table_name} (position, result, white, black, next_move)
        VALUES (%s, %s, %s, %s, %s);
        """
        cur.execute(insert_query, (row['Position'], row['Result'], row['White'], row['Black'], row['Next_Move']))

    # Commit changes
    conn.commit()   

# load data

In [5]:
def file_to_pandas(game):
    pass

In [6]:
folder_path = 'Final-merged'
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [7]:
for file in tqdm(files, desc="Processing files"):
    pgn_file_path = f'Final-merged/{file}'
    with open(pgn_file_path, "r") as pgn_file:
        games = []
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    data = []
    for game in games:
        result = game.headers.get("Result", "Unknown")
        white_player = game.headers.get("White", "Unknown")
        black_player = game.headers.get("Black", "Unknown")
        board = game.board()
        for move in game.mainline_moves():
            current_position = str(board)
            board.push(move)
            next_move = move.uci()
            data.append({
                "Position": current_position,
                "Result": result,
                "White": white_player,
                "Black": black_player,
                "Next_Move": next_move
            })
    save_data_to_db(data)
    

Processing files: 100%|███████████████████████| 294/294 [44:41<00:00,  9.12s/it]
